In [7]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import animation
plt.rcParams['animation.ffmpeg_path'] = r"C:\Users\rfgla\Documents\Ray\ffmpeg-master-latest-win64-gpl\ffmpeg-master-latest-win64-gpl\bin\ffmpeg.exe"
from IPython.display import HTML

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
from torchvision.io import read_video, read_video_timestamps
import sys
sys.path.append(r'C:\Users\rfgla\Documents\Ray\telerehab_exercise_feedback\VideoGPT-master')
from videogpt import download, VQVAE, load_videogpt
from videogpt.data import preprocess

In [8]:
# Load the VQ-VAE
filepath = r"C:\Users\rfgla\Documents\Ray\telerehab_exercise_feedback\VideoGPT-master\lightning_logs\version_2\checkpoints\epoch=43-step=136135.ckpt"
device = torch.device('cuda')
vqvae = VQVAE.load_from_checkpoint(filepath).to(device)

In [9]:
# Load the video
# `resolution` must be divisible by the encoder image stride
# `sequence_length` must be divisible by the encoder temporal stride
resolution, sequence_length = vqvae.args.resolution, 16

video_filename = r"C:\Users\rfgla\Documents\Ray\telerehab_exercise_feedback\data\gesture_sorted_data_resized\test\EFL\102_18_0_9_1_chair.mp4"
pts = read_video_timestamps(video_filename, pts_unit='sec')[0]
video = read_video(video_filename, pts_unit='sec', start_pts=pts[0], end_pts=pts[sequence_length - 1])[0]
video = preprocess(video, resolution, sequence_length).unsqueeze(0).to(device)
print(video.shape)

torch.Size([1, 3, 16, 64, 64])


In [10]:
# Apply the VQ-VAE to get an encoding and reconstruct the video
with torch.no_grad():
    encodings = vqvae.encode(video)
    video_recon = vqvae.decode(encodings)
    video_recon = torch.clamp(video_recon, -0.5, 0.5)


In [11]:
# Visualize the reconstruction
print(video.shape)
print(video_recon.shape)
videos = torch.cat((video, video_recon), dim=-1)
videos = videos[0].permute(1, 2, 3, 0)  # CTHW -> THWC
videos = ((videos + 0.5) * 255).cpu().numpy().astype('uint8')
print(videos.shape)

fig = plt.figure()
plt.title('real (left), reconstruction (right)')
plt.axis('off')
im = plt.imshow(videos[0, :, :, :])
plt.close()

def init():
    im.set_data(videos[0, :, :, :])

def animate(i):
    im.set_data(videos[i, :, :, :])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=videos.shape[0], interval=200) # 200ms = 5 fps
HTML(anim.to_html5_video())

torch.Size([1, 3, 16, 64, 64])
torch.Size([1, 3, 16, 64, 64])
(16, 64, 128, 3)
